Install d4rl-pybullet.

In [ ]:
!pip install git+https://github.com/takuseno/d4rl-pybullet

Setup dataset.

In [ ]:
import gym
import d4rl_pybullet

# setup environment including dataset
env = gym.make('hopper-bullet-mixed-v0')

# extract dataset
data = env.get_dataset()

Setup data-driven deep reinforcement learning.

In [ ]:
from d3rlpy.dataset import MDPDataset
from d3rlpy.algos import CQL
from d3rlpy.metrics.scorer import discounted_sum_of_advantage_scorer
from d3rlpy.metrics.scorer import evaluate_on_environment
from d3rlpy.metrics.scorer import td_error_scorer
from d3rlpy.metrics.scorer import average_value_estimation_scorer
from sklearn.model_selection import train_test_split

# setup MDPDataset
dataset = MDPDataset(data['observations'], data['actions'], data['rewards'], data['terminals'])

# this can be replaced with the builtin helper function as follows:
# from d3rlpy.datasets import get_pybullet
# dataset, env = get_pybullet('hopper-bullet-mixed-v0')

# setup CQL algorithm
cql = CQL(use_gpu=True)

# split train and test episodes
train_episodes, test_episodes = train_test_split(dataset, test_size=0.2)

# start training
cql.fit(train_episodes,
        eval_episodes=test_episodes,
        n_epochs=10,
        scorers={
            'environment': evaluate_on_environment(env), # use d4rl-pybullet environment.
            'advantage': discounted_sum_of_advantage_scorer, # smaller is better
            'td_error': td_error_scorer, # smaller is better
            'value_scale': average_value_estimation_scorer # smaller is better
        })